[ 필수과제 2 데이터 공유 ]
- movie_rv.csv 네이버 리뷰 데이터 0은 부정, 1은 긍정
- 긍부정에 따른 전처리 진행
- 부정 리뷰 중 유사도가 높은, 긍정 리뷰중 유사도가 높은 리뷰들 확인해 보기
- 랜덤 발표 진행 예정

In [16]:
import pandas as pd
movie = pd.read_csv('movie_rv.csv')
movie = movie[['id', 'document', 'label']]

# 데이터 양이 너무 크기 때문에
movie = movie[:500]

In [17]:
movie

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
495,10049872,그냥 기독교영화네요. 좀 더 깊이있는 내용을 기대했는데.. 실망입니다. 영화도 뭔가...,0
496,9427493,그냥 책으로 읽는게 더 절절하게 다가온다. 영화는 내용을 너무 비약하고 삭제해서 행...,0
497,6629097,너무나 감동적인 영화,1
498,9995177,스킨헤드성님들이 이 영화를 싫어합니다.,0


# 전처리

In [24]:
# 한글만 남기기
movie['document'] = movie['document'].str.replace('[^ㄱ-ㅣ가-힣\s]',"", regex=True)  #regex
# 긍부정 분류
pos = movie[movie['label']==1] # 긍정
neg =  movie[movie['label']==0] # 부정

# 형태소 분석

In [ ]:
!pip install konlpy
from konlpy.tag import Okt
from konlpy.tag import Twitter

In [27]:
twt = Twitter()
# 긍정데이터만
parts =['Noun']
pos_words_df = pd.DataFrame()
satisfaction = []
for n in range(len(pos)):
    text = pos['document'].iloc[n]
    words = twt.pos(text)
    words_df = pd.DataFrame()
    for i in words:
        if i =='EOS' or i =='':continue
        word_tmp = i[0]
        part = i[1]
        if not (part in parts):continue
        words_df[word_tmp]=[1] #형태소로 나눈 것들은 다 1로 넣어 달라는 것
    pos_words_df = pd.concat([pos_words_df, words_df],ignore_index =True)

/usr/local/lib/python3.10/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [31]:
all_words = []
parts= ['Noun']
for i in range(len(pos)):
    text = pos['document'].iloc[i]
    words = twt.pos(text)
    words_arr =[]
    for n in words:
        if n =='EOS' or n =='':continue
        words_tmp=n[0]
        part = n[1]
        if not (part in parts):continue
        words_arr.append(words_tmp)
    all_words.extend(words_arr)
print(all_words)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '몬페', '의', '연기', '영화', '스파이더맨', '커스틴', '던스트', '액션', '재미', '몇', '영화', '왜케', '평점', '꽤', '볼', '데', '헐리우드', '인피니트', '짱', '진짜', '볼때', '년대', '향수', '허진호', '감성', '절제', '멜로', '달인', '로만', '자꾸', '그', '사람', '것', '냥', '매번', '긴장', '재밋음', '사람', '바스코', '이기', '락스', '코', '바비', '이기', '아이돌', '다그', '안달', '것', '이건', '정말', '깨알', '캐스팅', '질퍽', '용구성', '버무러진', '깨알', '일드', '약탈', '위', '변명', '놈', '놈', '절대', '걸', '나름', '뜻', '듯', '그냥', '학생', '선생', '영화', '절대', '건', '절대', '영화', '수작', '는걸', '고추', '재밋는뎅', '센스', '연출', '캐스팅', '년대', '향수', '점', '엄포스', '위력', '다시', '한번', '적남', '꽃', '검사', '연기', '정말', '완전', '명품', '드라마', '점', '왜', '이', '드라마', '인생', '최고', '패션', '대한', '열정', '안나', '윈', '투어', '작가', '정신', '유령', '이', '영화', '왜', '평가', '매력', '영화', '서리', '이', '정말', '맘', '또', '또', '방법', '윤제문', '배우', '발견', '탈', '미소', '머금', '음악', '조금', '점', '평점', '점', '데', '리스', '타르', '엔나', '용의', '주인', '누', '근친상간', '소설', '속', '제일', '놈', '자', '메', '니스', '터', '드라마', '속', '드래곤', '용이', '제일', '웃음', '감독', '토르', '다크',

In [40]:
#카운팅
all_words_df = pd.DataFrame({'words':all_words, 'count':len(all_words)*[1]})
all_words_df.groupby('words').sum()
words_df1= all_words_df.groupby('words').sum()
words_df1.sort_values('count',ascending = False).head(10)
pos_word_list = words_df1.sort_values('count',ascending = False).head(10).index.to_list()
print(pos_word_list)

['영화', '정말', '연기', '최고', '이', '감동', '보고', '더', '말', '드라마']


In [28]:
# 부정데이터만
parts =['Noun']
neg_words_df = pd.DataFrame()
satisfaction = []
for n in range(len(neg)):
    text = neg['document'].iloc[n]
    words = twt.pos(text)
    words_df = pd.DataFrame()
    for i in words:
        if i =='EOS' or i =='':continue
        word_tmp = i[0]
        part = i[1]
        if not (part in parts):continue
        words_df[word_tmp]=[1] #형태소로 나눈 것들은 다 1로 넣어 달라는 것
    neg_words_df = pd.concat([neg_words_df, words_df],ignore_index =True)

In [41]:
all_words = []
parts= ['Noun']
for i in range(len(neg)):
    text = neg['document'].iloc[i]
    words = twt.pos(text)
    words_arr =[]
    for n in words:
        if n =='EOS' or n =='':continue
        words_tmp=n[0]
        part = n[1]
        if not (part in parts):continue
        words_arr.append(words_tmp)
    all_words.extend(words_arr)
print(all_words)

['더빙', '진짜', '목소리', '무재', '밓었', '다그', '래서', '추천', '교도소', '이야기', '구먼', '재미', '평점', '조정', '막', '걸음', '마', '세', '초등학교', '학년', '생인', '영화', '반개', '움', '원작', '긴장감', '제대로', '별', '반개', '욕', '이응경', '길용우', '생활', '년', '정말', '발', '해도', '그것', '납치', '감금', '반복', '반복', '드라마', '가족', '연기', '사람', '네', '울면', '손', '횡단보도', '때', '뻔', '이범수', '연기', '취향', '존중', '다지', '진짜', '내생', '극장', '영화', '가장', '노잼', '노', '감동', '임', '스토리', '어거지', '감동', '어거지', '굿바이', '레닌', '표절', '것', '이해', '왜', '뒤', '갈수록', '재미', '음식', '영화', '도', '바베트', '만찬', '차이', '남바', '베트', '만찬', '이야기', '음식', '재미', '이건', '볼', '음식', '별로', '핀란드', '풍경', '할랫', '그것', '별로', '주제', '중반', '짤랐을꺼', '납득', '수', '꼭', '카밀라', '벨', '발연기', '졸', '쓰레기', '도안', '아시', '내', '죄인', '죄인', '키이라', '나이틀리', '연기', '대체', '정신장애', '틱장애', '포스터', '관객', '명', '알바생', '내용', '하나', '완전', '별', '영화', '내', '시간', '움', '완전', '낚임', '평점', '속지', '시간', '낭비', '돈', '낭비', '임', '리얼리티', '공감', '안', '간다', '이민기', '캐릭터', '정신의학', '분노조절', '장애', '초기', '증상', '툭하면', '사람', '패', '욕', '물건', '파손', '조금', '바', '초반', '가면', '갈수록',

In [42]:
#카운팅
all_words_df = pd.DataFrame({'words':all_words, 'count':len(all_words)*[1]})
all_words_df.groupby('words').sum()
words_df1= all_words_df.groupby('words').sum()
words_df1.sort_values('count',ascending = False).head(10)
neg_word_list = words_df1.sort_values('count',ascending = False).head(10).index.to_list()
print(neg_word_list)

['영화', '진짜', '왜', '점', '내용', '연기', '주인공', '별로', '재미', '정말']


# 긍정

In [29]:
print(pos_words_df)
pos_words_df1 = pos_words_df.fillna(0)
pos_words_df1

       흠  포스터   보고   초딩   영화    줄   오버   연기   몬페    의  ...  할아버지  상술  선거   술  \
0    1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  NaN  NaN  ...   NaN NaN NaN NaN   
1    NaN  NaN  NaN  NaN  1.0  NaN  NaN  1.0  1.0  1.0  ...   NaN NaN NaN NaN   
2    NaN  NaN  NaN  NaN  1.0  NaN  NaN  NaN  NaN  NaN  ...   NaN NaN NaN NaN   
3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN NaN NaN NaN   
4    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN NaN NaN NaN   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  ..  ..  ..   
243  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN NaN NaN NaN   
244  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN NaN NaN NaN   
245  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN NaN NaN NaN   
246  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN NaN NaN NaN   
247  NaN  NaN  NaN  NaN  1.0  NaN  NaN  NaN  NaN  NaN  ...   NaN NaN NaN NaN   

     쓰레기  함부로   맙시   줄리   델피   쵝오  
0  

,흠,포스터,보고,초딩,영화,줄,오버,연기,몬페,의,...,할아버지,상술,선거,술,쓰레기,함부로,맙시,줄리,델피,쵝오
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
target_text = pos_words_df1.iloc[1]
for i in range(len(target_text)):
  if target_text.index[i] in pos_word_list :
    target_text[i]= 1.0
  else :
    target_text[i]= 0.0


<ipython-input-55-c40534c11ce6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_text[i]= 0.0
<ipython-input-55-c40534c11ce6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_text[i]= 1.0


In [56]:
import numpy as np
cos_sim=[]
for i in range(len(pos_words_df1)):
    cos_text = pos_words_df1.iloc[i]
    cos = np.dot(target_text, cos_text) / (np.linalg.norm(target_text)*np.linalg.norm(cos_text))
    cos_sim.append(cos)

pos_words_df1['cos_sim'] = cos_sim

In [57]:
pos_words_df1.sort_values('cos_sim',ascending=False).head(10)

,흠,포스터,보고,초딩,영화,줄,오버,연기,몬페,의,...,상술,선거,술,쓰레기,함부로,맙시,줄리,델피,쵝오,cos_sim
89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.547723
235,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.543427
198,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.543427
97,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.543427
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.474342
247,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.439435
226,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.439435
199,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.439435
103,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.439435
86,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.439435


# 결과 확인

In [59]:
print(pos_word_list)
pos.iloc[89]

['영화', '정말', '연기', '최고', '이', '감동', '보고', '더', '말', '드라마']


id                                    9169160
document    자극적인 것에 익숙해진 현대인이 봐도 눈을 떼기 힘든 연출력
label                                       1
Name: 178, dtype: object

In [62]:
pos.iloc[235]

id           5351337
document    감동적인 영화다
label              1
Name: 454, dtype: object

# 부정

In [30]:
print(neg_words_df)
neg_words_df1 = neg_words_df.fillna(0)
neg_words_df1

      더빙   진짜  목소리   무재   밓었   다그   래서   추천  교도소  이야기  ...   삭제   행간    꽉  \
0    1.0  1.0  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
1    NaN  NaN  NaN  1.0  1.0  1.0  1.0  1.0  NaN  NaN  ...  NaN  NaN  NaN   
2    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  ...  NaN  NaN  NaN   
3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
4    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
230  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
231  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
232  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  1.0  1.0  1.0   
233  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
234  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   

      반대  스킨헤드    성  자동차  매니아   이하    았  
0    NaN   NaN  NaN  NaN  NaN  Na

,더빙,진짜,목소리,무재,밓었,다그,래서,추천,교도소,이야기,...,삭제,행간,꽉,반대,스킨헤드,성,자동차,매니아,이하,았
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [60]:
target_text = neg_words_df1.iloc[1]
for i in range(len(target_text)):
  if target_text.index[i] in neg_word_list :
    target_text[i]= 1.0
  else :
    target_text[i]= 0.0

In [61]:
import numpy as np
cos_sim=[]
for i in range(len(neg_words_df1)):
    cos_text = neg_words_df1.iloc[i]
    cos = np.dot(target_text, cos_text) / (np.linalg.norm(target_text)*np.linalg.norm(cos_text))
    cos_sim.append(cos)

neg_words_df1['cos_sim'] = cos_sim

In [63]:
neg_words_df1.sort_values('cos_sim',ascending=False).head(10)

,더빙,진짜,목소리,무재,밓었,다그,래서,추천,교도소,이야기,...,행간,꽉,반대,스킨헤드,성,자동차,매니아,이하,았,cos_sim
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.447214
155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.365148
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.365148
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.365148
215,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.358569
219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.335410
77,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.335410
60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.316228
226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.316228


# 결과 확인

In [70]:
print(neg_word_list)
neg.iloc[204].document

['영화', '진짜', '왜', '점', '내용', '연기', '주인공', '별로', '재미', '정말']


'이딴게 한국 영화라니'

In [68]:
neg.iloc[155].document

'지금 이거 티비로 돈내고 보고있는 내가한심하다'